In [2]:
import pinocchio as pin
import numpy as np
from FlexivPy.robot.model.pinocchio import FlexivModel
from FlexivPy.sim.MuJoCo import FlexivSimMujoco as FlexivSim
from FlexivPy.robot.dds.flexiv_messages import FlexivCmd

In [3]:
model = FlexivModel()
robot = FlexivSim(render=True, dt=0.002, mode="position")

robot sim is ready!


In [4]:
from FlexivPy.controllers.taskspace import DiffIKController

task_controller = DiffIKController(model, dt=0.01)

## Direct Simulation

In [5]:
robot.reset_state_robot(model.q0, np.zeros_like(model.q0))
cmd = FlexivCmd()
cmd.q = model.q0
robot.set_cmd(cmd)
robot.viewer.sync()
s = robot.get_robot_state()
T_cmd = model.getInfo(model.q0, np.zeros(7))["poses"]["link7"].copy()
T_cmd[2, -1] += 0.1
q_des = s.q

In [6]:
import time

for i in range(5000):
    s = robot.get_robot_state()
    dq_des = task_controller(s.q, s.dq, T_cmd).squeeze()
    dt = robot.dt
    cmd = FlexivCmd()
    cmd.kp = np.ones(7) * 15 * 0
    cmd.kv = np.ones(7) * 8
    q_des += dq_des * dt
    q_des = np.clip(q_des, robot.joints_lower_limits, robot.joints_upper_limits)
    cmd.q = q_des
    cmd.dq = dq_des
    robot.set_cmd(cmd)
    for _ in range(5):
        robot.step()

In [10]:
import time

s = robot.get_robot_state()
task_controller.setup(s)
for i in range(5000):
    s = robot.get_robot_state()
    cmd = task_controller.get_control(s, 0)
    robot.set_cmd(cmd)
    for _ in range(5):
        robot.step()

## Using The Runners

### Blocking

In [7]:
from FlexivPy.controllers.runners import blocking_runner

In [8]:
from FlexivPy.controllers.jointspace import RRTController

homing_controller = RRTController(control_mode="velocity")

In [9]:
blocking_runner(robot, homing_controller)

goal reached


<ControllerStatus.GOAL_REACHED: 1>

In [ ]:
robot.reset_state_robot(model.q0, np.zeros(7))
robot.viewer.sync()
blocking_runner(robot, task_controller)

### Nonblocking

In [4]:
from FlexivPy.controllers.runners import NonBlockingRunner

runner = NonBlockingRunner(robot, task_controller)

In [ ]:
runner.close()

In [ ]:
info = model.getInfo(model.q0, np.zeros(7))
T_cmd = info["poses"]["link7"]

In [ ]:
task_controller.T_cmd = T_cmd

In [ ]:
runner.close()